In [ ]:
import cv2
import numpy as np

In [23]:
video = cv2.VideoCapture(0)
video.set(3,1000)      #3 is id for length 
video.set(4,1000)      #4 is id for breadth  
video.set(10,200)     #10 is id for brightness

address = "http://192.168.137.206:8080/video"
video.open(address)

def getcontour(img,img_dis):
    contour,heirarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    for ctr in contour:
        peri = cv2.arcLength(ctr,True)
        approx = cv2.approxPolyDP(ctr,0.02*peri,True)
        sides = len(approx)
        area = cv2.contourArea(ctr)
        if sides==4 and area>1000:
            return approx
        else:
            pass
count = 0

while True:
    
    success , img_webcam = video.read()
    img_grey = cv2.cvtColor(img_webcam,cv2.COLOR_BGR2GRAY)
    img_blur = cv2.blur(img_grey,(3,3))
    img_canny = cv2.Canny(img_blur,150,150)
    
    points = getcontour(img_canny,img_webcam)
    points = np.array(points)
    
    
    if points.shape == (4,1,2):
        cv2.drawContours(img_webcam, points,-4,(100,0,200),20)
        points = np.reshape(points,(4,2)) 
        sum_arr = np.sum(points,axis=1)  #to check order
        diff_arr = np.diff(points,axis=1)
        final_arr = [points[np.argmin(sum_arr)],points[np.argmin(diff_arr)],
                     points[np.argmax(diff_arr)],points[np.argmax(sum_arr)]]
        
        width,height = 400,700
        pts1 = np.float32(final_arr)
        pts2 = np.float32([[0,0],[width,0],[0,height],[width,height]])

        matrix = cv2.getPerspectiveTransform(pts1,pts2)
        final_img = cv2.warpPerspective(img_webcam,matrix,(width,height))
        count += 1
        img_copy = final_img.copy()
        
    else:
        if count == 0 :
            final_img = np.ones((800,500,3))
        else:
            final_img = img_copy
    
    cv2.imshow("video",img_webcam)
    cv2.imshow("video ",final_img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        cv2.destroyAllWindows()
        break